In [28]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer,AutoConfig
from transformers import AutoModelForCausalLM
import torch
import evaluate
import re

#del model
#del trainer
torch.cuda.empty_cache()

In [29]:
def clean_text(text):
    # Satır sonlarını kaldır
    text = text.replace('\n', ' ')
    
    # Özel karakterleri kaldır
    text = re.sub(r'[^A-Za-z0-9\s\.?]', '', text)
    
    # Fazla boşlukları kaldır
    text = ' '.join(text.split())
    
    return text

In [30]:
with open("AROG-ENG-SUBTITLE.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    raw_text_c=clean_text(raw_text)
with open("Gora-ENG.txt", "r", encoding="utf-8") as f:
    raw_text_2 = f.read()
    raw_text_c_2=clean_text(raw_text_2)
with open("yahsi_bati_eng_sub-utf.txt", "r", encoding="utf-8") as f:
    raw_text_3 = f.read()
    raw_text_c_3=clean_text(raw_text_3)    
raw_text_c=raw_text_c+raw_text_c_2+raw_text_c_3
print(len(raw_text_c))

177826


In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
raw_data={"train":raw_text_c.split("."), 
         "test":raw_text_c[130000:].split(".")}

#print(raw_data["train"])
def preprocess_function(examples,tokenizer=tokenizer):
    tokenize_data=[]
    for example in examples:
        tokenize_data.append(tokenizer(example))
    
    return tokenize_data
    
tokenize_data_train=preprocess_function(raw_data["train"])
tokenize_data_test=preprocess_function(raw_data["test"])
              


In [32]:
def convert_to_dict(data):
    return {key: [d[key] for d in data] for key in data[0]}

train_dict = convert_to_dict(tokenize_data_train)
test_dict = convert_to_dict(tokenize_data_test)

# 'Dataset' nesnelerini oluşturma
train_dataset = Dataset.from_dict(train_dict)
test_dataset = Dataset.from_dict(test_dict)

# 'DatasetDict' nesnesini oluşturma
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# Dataset özniteliklerini yazdırma
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 6163
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1672
    })
})


In [33]:
def group_texts(examples,block_size=128):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/6163 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1672 [00:00<?, ? examples/s]

In [34]:
print(lm_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 304
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 81
    })
})


In [35]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [36]:
training_args = TrainingArguments(
    output_dir="my_awesome_cmylmz_clm-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs= 200.0
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,4.742163
2,No log,4.484071
3,No log,4.307119
4,No log,4.189540
5,No log,4.080228
6,No log,3.988657
7,No log,3.909014
8,No log,3.838206
9,No log,3.768127
10,No log,3.693084


TrainOutput(global_step=7600, training_loss=1.0210297659823768, metrics={'train_runtime': 33920.9774, 'train_samples_per_second': 1.792, 'train_steps_per_second': 0.224, 'total_flos': 3971638886400000.0, 'train_loss': 1.0210297659823768, 'epoch': 200.0})

In [37]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.79


In [44]:
from transformers import pipeline
prompt="Arif"
tokenizer =tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
generator = pipeline("text-generation", model="baloglu321/my_awesome_cmylmz_clm-model",tokenizer=tokenizer)
generator(prompt)





Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Arif carpet rug travel Damn l wish l had my bussiness card with me Look l have gum Put this away put it He is obviously from Arogan Trying to spy on us The lady rocks by the way must be because of the Age'}]